In [1]:
import json
import os
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
from statistics import mean,variance

In [2]:
directory = '/Users/shashankgupta/Desktop/PRAW-Data/New2/All/'
file_list = os.listdir(directory)
json_files = [pos_json for pos_json in file_list if pos_json.endswith('.json')]

In [3]:
df = pd.DataFrame(pd.read_json(directory + json_files[0]))
  
for i in range(1,len(json_files)):
    data = pd.read_json(directory + json_files[i])
    df = pd.concat([df,data],axis=0,ignore_index=True)

In [4]:
def get_context(text, w, left = 10, right = 10):
    w = w[0].strip().split()[0]
    
    w = w.replace("(","\(")
    w = w.replace(")","\)")
    w = w.replace("[","\[")
    w = w.replace("]","\]")
    w = w.replace("*","\*")
    w = w.replace("+","\+")
    w = w.replace("-","\-")
    w = w.replace("{","\{")
    w = w.replace("}","\}")
    
    all_contexts = []
    for i in range(len(text)):
        if re.findall(w,text[i]):
            if i < left:
                context_left = text[:i]
            else:
                context_left = text[i-left:i]
            if len(text) < (i+right):
                context_right = text[i:]
            else:
                context_right = text[i:(i+right+1)]
            context = context_left + context_right
            all_contexts.append(context)
    return all_contexts

def find_gender(regex,text):
    gender = re.findall(regex,text)
    return gender

def generate_N_grams(words,ngram=2):
    temp=zip(*[words[i:] for i in range(0,ngram)])
    ans=[' '.join(ngram) for ngram in temp]
    return ans

def find_race(context):
    race = "unknown"
    lat_his_race = ["latino", "hispanic", "latina", "latinx", "mexican", "cuban", "puerto rican"]
    race_white = ["caucasian","white", "ww", "wm", "european"]
    black_race = ["black", "african american", "aa", "bw", "bm", "afro-american", "afro", "afro american"]
    asian_race = ["asian"]
    native_american = ["native american", "american indian", "indigenous", "alaska native" ]
    hawaiin_race = ["hawaiin", "pacific islander"]
    
    for token in context:
        token = re.sub(r'[^\w\s]', '', token)
        if token in lat_his_race:
            race = "latino/hispanic"
            break
            
    if race == "unknown":
        for token in context:
            token = re.sub(r'[^\w\s]', '', token)
            if token in race_white:
                race = "white"
                break

            elif token in black_race:
                race = "black"
                break

            elif token in asian_race:
                race = "asian"
                break

            elif token in native_american:
                race = "american indian"
                break

            elif token in hawaiin_race:
                race = "hawaiin"
                break
            
    return race

def calculate_mean(data):
    avg = 0
    if len(data)!= 0:
        avg = mean(data)
    
    return avg 

def preprocess_text(text):
    text = text.lower()
    text = text.replace("\n", " ")
    text = text.replace("\t", " ")
    text = text.replace("[", " ")
    text = text.replace("]", " ")
    text = text.replace("/", " ")
    text = text.replace("\\", " ")
    text = text.replace("(", " ")
    text = text.replace(")", " ")
    text = text.replace("{", " ")
    text = text.replace("}", " ")
    
    return text

class color:
    PURPLE = '\033[95m'
    CYAN = '\033[96m'
    DARKCYAN = '\033[36m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'
    
def calculate_average(a, b):
    avg = 0
    if b != 0:
        avg = round(a/b,3)
        
    return avg

def get_age(context,w):
    
    punctuation = '!"#$%&\'()*+,.-/:;<=>?@[\\]^_`{|}~'
    context = [''.join(letter for letter in word if letter not in punctuation) for word in context]
    context = [word for word in context if word not in stopwords.words('english')]
        
    w = w[0].strip()
    w = w.replace("(","\(")
    w = w.replace(")","\)")
    w = w.replace("[","\[")
    w = w.replace("]","\]")
    w = w.replace("*","\*")
    w = w.replace("+","\+")
    w = w.replace("-","\-") 
    w = w.replace("{","\{")
    w = w.replace("}","\}")
    w = w.replace(",","")
    
#     print(w,context)
      
    age = "unknown"
    
    age_list = re.findall("\d{2}", w)
    if len(age_list)!= 0:
        age = age_list[0]
        
    elif len(context)!= 0 and age == "unknown":
        for tokens in context:
            age_list2 = re.findall("(^\d\d[\s|-]?yr|^\d\d[\s|-]?yrs|^\d\d[\s|-]?year|^\d\d[\s|-]?years|^\d\d[\s|-]?yo|^\d\d[\s|-]?ys|^\d\d[\s|-]?y)",tokens.strip())
            if len(age_list2)!= 0:
                age_string = age_list2[0]
                if age_string:
                    age_s = re.findall("\d{2}", age_string)
                    if age_s:
                        age= age_s[0]
                        break
                        
            elif age == "unknown":
                age_list3 = re.findall("age[\s:-]?\d\d",tokens.strip())
                if len(age_list3) != 0:
                    age_string3 = age_list3[0]
                    if age_string3:
                        age_s3 = re.findall("\d{2}", age_string3)
                        if age_s3:
                            age = age_s3[0]
                            break
                            
    if age == "unknown":
        if w in context:
            index = context.index(w)
            
            left_index = index - 1
            right_index = index + 1
            
            if left_index > -1:
                age_s4 = re.findall("\d{2}", context[left_index])
                if age_s4:
                    age = age_s4[0]
            
            if right_index < len(context) and age == "unknown":
                age_s5 = re.findall("\d{2}", context[right_index])
                if age_s5:
                    age = age_s5[0]
                    
    return age

def get_age_without_context(selftext,title,age):
    
    punctuation = '!"#$%&\'()*+,.-/:;<=>?@[\\]^_`{|}~'
    selftext = [''.join(letter for letter in word if letter not in punctuation) for word in selftext]
    selftext_2gram = generate_N_grams(selftext,2)
    selftext_3gram = generate_N_grams(selftext,3)
    selftext_2gram.extend(selftext_3gram)
    
    for tokens in selftext_2gram:
        age_list2 = re.findall("(^\d\d[\s|-]?yr old|^\d\d[\s|-]?yrs old|^\d\d[\s|-]?year old|^\d\d[\s|-]?years old|^\d\d[\s|-]?yo|^\d\d[\s|-]?ys|^\d\d[\s|-]?y)",tokens.strip())
        if len(age_list2)!= 0:
            age_string = age_list2[0]
            if age_string:
                age_s = re.findall("\d{2}", age_string)
                if age_s:
                    age= age_s[0]
                    break
                    
        if age == "unknown":
            age_list3 = re.findall("age[\s:-]?\d\d",tokens.strip())
            if len(age_list3) != 0:
                age_string3 = age_list3[0]
                if age_string3:
                    age_s3 = re.findall("\d{2}", age_string3)
                    if age_s3:
                        age = age_s3[0]
                        break
                        
    if age == "unknown":
        title = [''.join(letter for letter in word if letter not in punctuation) for word in title]
        title_2gram = generate_N_grams(title,2)
        title_3gram = generate_N_grams(title,3)
        title_2gram.extend(title_3gram)

        for tokens in title_2gram:
            age_list2 = re.findall("(^\d\d[\s|-]?yr old|^\d\d[\s|-]?yrs old|^\d\d[\s|-]?year old|^\d\d[\s|-]?years old|^\d\d[\s|-]?yo|^\d\d[\s|-]?ys|^\d\d[\s|-]?y)",tokens.strip())
            if len(age_list2)!= 0:
                age_string = age_list2[0]
                if age_string:
                    age_s = re.findall("\d{2}", age_string)
                    if age_s:
                        age= age_s[0]
                        break

            elif age == "unknown":
                age_list3 = re.findall("age[\s:-]?\d\d",tokens.strip())
                if len(age_list3) != 0:
                    age_string3 = age_list3[0]
                    if age_string3:
                        age_s3 = re.findall("\d{2}", age_string3)
                        if age_s3:
                            age = age_s3[0]
                            break
                
    return age

In [5]:
# race lists
lat_his_cmnts_list = []
white_cmnts_list = []
black_cmnts_list = []
asian_cmnts_list = []
american_indian_cmnts_list = []
hawaiin_cmnts_list = []
unknown_race_cmnts_list = []

verified_lat_his_cmnts_list = []
verified_white_cmnts_list = []
verified_black_cmnts_list = []
verified_asian_cmnts_list = []
verified_american_indian_cmnts_list = []
verified_hawaiin_cmnts_list = []
verified_unknown_race_cmnts_list = []

lat_his_OP_list = []
white_OP_list = []
black_OP_list = []
asian_OP_list = []
american_indian_OP_list = []
hawaiin_OP_list = []
unknown_race_OP_list = []

lat_his_engagement = []
white_engagement = []
black_engagement = []
asian_engagement = []
american_indian_engagement = []
hawaiin_engagement = []
unknown_race_engagement = []

lat_his_engagement_post_count = []
white_engagement_post_count = []
black_engagement_post_count = []
asian_engagement_post_count = []
american_indian_engagement_post_count = []
hawaiin_engagement_post_count = []
unknown_race_engagement_post_count = []

#gender lists
male_comnt_list = []
female_comnt_list = []
trans_comnt_list = []
binary_comnt_list = []
unknown_cmnt_list = []

verified_male_list = []
verified_female_list = []
verified_trans_list = []
verified_nbinary_list = []
verified_unknown_list = []

male_OP_list = []
female_OP_list = []
trans_OP_list = []
nbinary_OP_list = []
unknown_OP_list = []

male_engagement = []
female_engagement = []
non_binary_engagement = []
trans_engagement = []
unknown_engagement = []

male_engagement_post_count = []
female_engagement_post_count = []
non_binary_engagement_post_count = []
trans_engagement_post_count = []
unknown_engagement_post_count = []

# age lists
child_comnt_list = []
teen_comnt_list = []
adult_comnt_list = []
middle_age_comnt_list = []
senior_cmnt_list = []
unknown_age_cmnt_list = []

verified_child_cmnt_list = []
verified_teen_cmnt_list = []
verified_adult_cmnt_list = []
verified_middle_age_cmnt_list = []
verified_senior_cmnt_list = []
verified_unknown_age_cmnt_list = []

child_OP_list = []
teen_OP_list = []
adult_OP_list = []
middle_age_OP_list = []
senior_OP_list = []
unknown_age_OP_list = []

child_engagement = []
teen_engagement = []
adult_engagement = []
middle_age_engagement = []
senior_engagement = []
unknown_age_engagement = []

child_engagement_post_count = []
teen_engagement_post_count = []
adult_engagement_post_count = []
middle_age_engagement_post_count = []
senior_engagement_post_count = []
unknown_age_engagement_post_count = []


post_num_atleast_one_nonOP_cmnt = []
zero_cmnts_post = []
atleast_one_prof_nonOP_cmnt = []

#posts having zero comments (testing)
jiro = []
abc = []
total_posts = []

# sets to count unique type of users
unique_posters = set()
unique_commenters = set()
unique_professionals = set()

for i in range(len(df)):
    
    gender = "unknown"
    race = "unknown"
    age = "unknown"
    
#     print('\n')
#     print(i)
    
    # count of unique posters
    total_posts.append(i)
    unique_posters.add(df['post_author'][i])
    
    selftext = preprocess_text(df['selftext'][i])
    title = preprocess_text(df['title'][i])
    
#     print(color.RED + "title : " + color.END, title)
#     print(color.GREEN + "selftext : " + color.END, selftext)
    
    #tokenization
    title_split = title.split()
    selftext_split = selftext.split()
    
    trans_regex = "(?i)trans[\s-]*(?i)gender|(?i)trans[\s-]*(?i)man|(?i)trans[\s-]*(?i)woman|(?i)trans[\s-]*(?i)male|(?i)trans[\s-]*(?i)female|(?i)trans[\s-]*(?i)masculine|(?i)trans[\s-]*(?i)feminine|(?i)trans[\s-]*(?i)guy|(?i)trans[\s-]*(?i)girl|(?i)[MF](?i)[2t](?i)[MF]|(?i)trans[\s.\)]"
    trans_title  = find_gender(trans_regex,title)
    trans_selftext  = find_gender(trans_regex,selftext)
    
    female_regex = "(?i)female|(?i)woman"
    female_title   = find_gender(female_regex,title )
    female_selftext  = find_gender(female_regex,selftext)
    
    male_regex  = "(?i)(?<!fe)male|(?i)(?<![a-z])man[\s.,;()\n/-]|(?i)guy(?![s])[\s.,;()\n/-]"
    male_title  = find_gender(male_regex,title )
    male_selftext  = find_gender(male_regex,selftext)
    
    y1_regex = "(?<!\.)[^\d]\d\d[\s.,;:/(\[-]*(?i)[(]*[mf][)\\\]]*[^a-zA-Z][\s]*"
    y1_title  = find_gender(y1_regex,title )
    y1_selftext  = find_gender(y1_regex,selftext)
    
    test_regex = "(?<![a-z\'\’\.])\d\d[\s.,;:/\\\(\[-\]]*(?i)[mf](?![a-z])|(?i)(?<![a-z\'\’])[mf][\s.,;:/\(-]*\d\d(?![a-z0-9])"
    test_title  = find_gender(test_regex,title )
    test_selftext  = find_gender(test_regex,selftext)
    
    f1_regex  = "\d\d[\s]*[a-zA-Z]+:\s(?i)[mf](?![a-z])"
    f1_title  = find_gender(f1_regex,title )
    f1_selftext  = find_gender(f1_regex,selftext)
    
    y243_regex   = "(?<!\.)\d\d[\s]*[.,;:/]*[a-zA-Z]*[\s]*[a-zA-Z]*[\s](?i)[(]*[mf][)\]]*[^a-zA-Z][\s]*"
    y243_title  = find_gender(y243_regex,title )
    y243_selftext  = find_gender(y243_regex,selftext)
    
    y666_regex   = "(?i)sex[\s:;\[\]\(\)/-]*[mf](?![a-z])"
    y666_title  = find_gender(y666_regex,title )
    y666_selftext  = find_gender(y666_regex,selftext)
    
    binary_regex = "(?i)non[\s-]?(?i)binary|(?i)gender[\s-]?(?i)queer|agender|(?i)bi[\s-]?(?i)gender|(?i)gender[\s-]?(?i)fluid|(?i)dual[\s-]?(?i)gender|androgyne|(?i)demi[\s-]?(?i)boy|(?i)demi[\s-]?(?i)girl|(?i)demi[\s-]?(?i)gender"
    binary_title  = find_gender(binary_regex,title )
    binary_selftext  = find_gender(binary_regex,selftext)
    
    special_solo_regex = "(?<![A-Za-z0-9])(?i)(m|f)[^A-Za-z][\s.,;\(\)\n/-][^A-Za-z][\d\d]"
    solo_title = find_gender(special_solo_regex,title )
    solo_selftext  = find_gender(special_solo_regex,selftext)
    
    trans, binary, female, male1, y1, testregex, f1, y243, y666, special_solo, context = [], [], [], [], [], [], [], [], [], [], []
    
    
    if len(trans_title)!=0 or len(trans_selftext)!=0 :
        if len(trans_title) !=0  and race == 'unknown':
            trans = trans_title
            context = get_context(title_split,trans_title)[0]
            context.extend(generate_N_grams(context,2))
            if age == "unknown":
                age = get_age(context,trans_title)
            race = find_race(context)
    
        if len(trans_selftext) != 0  and race == 'unknown':
            if not trans:
                trans = trans_selftext

            context = get_context(selftext_split,trans_selftext)[0]
            context.extend(generate_N_grams(context,2))
            if age == "unknown":
                age = get_age(context,trans_selftext)
            race = find_race(context)
            
    if len(binary_title)!=0 or len(binary_selftext)!=0 :
        if len(binary_title) !=0  and race == 'unknown':
            binary = binary_title
            context = get_context(title_split,binary_title)[0]
            context.extend(generate_N_grams(context,2))
            if age == "unknown":
                age = get_age(context,binary_title)
            race = find_race(context)
    
        if len(binary_selftext) != 0  and race == 'unknown':
            if not binary:
                binary = binary_selftext
            context = get_context(selftext_split,binary_selftext)[0]
            context.extend(generate_N_grams(context,2))
            if age == "unknown":
                age = get_age(context,binary_selftext)
            race = find_race(context)
        
    if len(female_title)!=0 or len(female_selftext)!=0 :
        if len(female_title) !=0 and race == 'unknown':
            female = female_title
            context = get_context(title_split,female_title)[0]
            context.extend(generate_N_grams(context,2))
            if age == "unknown":
                age = get_age(context,female_title)
            race = find_race(context)
    
        if len(female_selftext) != 0  and race == 'unknown':
            if not female:
                female = female_selftext
                
            context = get_context(selftext_split,female_selftext)[0]
            context.extend(generate_N_grams(context,2))
            if age == "unknown":
                age = get_age(context,female_selftext)
            race = find_race(context)
                
    if len(male_title)!=0 or len(male_selftext)!=0 :
        if len(male_title) !=0  and race == 'unknown':
            male1 = male_title
            context = get_context(title_split,male_title)[0]
            context.extend(generate_N_grams(context,2))
            if age == "unknown":
                age = get_age(context,male_title)
            race = find_race(context)
    
        if len(male_selftext) != 0  and race == 'unknown':
            if not male1:
                male1 = male_selftext
                
            context = get_context(selftext_split,male_selftext)[0]
            context.extend(generate_N_grams(context,2))
            if age == "unknown":
                age = get_age(context,male_selftext)
            race = find_race(context)
                
    if len(y1_title)!=0 or len(y1_selftext)!=0 :
        if len(y1_title) !=0  and race == 'unknown':
            y1 = y1_title
            context = get_context(title_split,y1_title)[0]
            context.extend(generate_N_grams(context,2))
            if age == "unknown":
                age = get_age(context,y1_title)
            race = find_race(context)
    
        if len(y1_selftext) != 0  and race == 'unknown':
            if not y1:
                y1 = y1_selftext
            context = get_context(selftext_split,y1_selftext)[0]
            context.extend(generate_N_grams(context,2))
            if age == "unknown":
                age = get_age(context,y1_selftext)
            race = find_race(context)
            
    if len(test_title)!=0 or len(test_selftext)!=0 :
        if len(test_title) !=0  and race == 'unknown':
            testregex = test_title
            context = get_context(title_split,test_title)[0]
            context.extend(generate_N_grams(context,2))
            if age == "unknown":
                age = get_age(context,test_title)
            race = find_race(context)
    
        if len(test_selftext) != 0  and race == 'unknown':
            if not testregex:
                testregex = test_selftext
            context = get_context(selftext_split,test_selftext)[0]
            context.extend(generate_N_grams(context,2))
            if age == "unknown":
                age = get_age(context,test_selftext)
            race = find_race(context)
    
    if len(f1_title)!=0 or len(f1_selftext)!=0 :
        if len(f1_title) !=0  and race == 'unknown':
            f1 = f1_title
            context = get_context(title_split,f1_title)[0]
            context.extend(generate_N_grams(context,2))
            if age == "unknown":
                age = get_age(context,f1_title)
            race = find_race(context)
    
        if len(f1_selftext) != 0  and race == 'unknown':
            if not f1:
                f1 = f1_selftext
            context = get_context(selftext_split,f1_selftext)[0]
            context.extend(generate_N_grams(context,2))
            if age == "unknown":
                age = get_age(context,f1_selftext)
            race = find_race(context)
                
    if len(y243_title)!=0 or len(y243_selftext)!=0 :
        if len(y243_title) !=0  and race == 'unknown':
            y243 = y243_title
            context = get_context(title_split,y243_title)[0]
            context.extend(generate_N_grams(context,2))
            if age == "unknown":
                age = get_age(context,y243_title)
            race = find_race(context)
    
        if len(y243_selftext) != 0  and race == 'unknown':
            if not y243:
                y243 = y243_selftext
            context = get_context(selftext_split,y243_selftext)[0]
            context.extend(generate_N_grams(context,2))
            if age == "unknown":
                age = get_age(context,y243_selftext)
            race = find_race(context)
    
    if len(y666_title)!=0 or len(y666_selftext)!=0 :
        if len(y666_title) !=0  and race == 'unknown':
            y666 = y666_title
            context = get_context(title_split,y666_title)[0]
            context.extend(generate_N_grams(context,2))
            if age == "unknown":
                age = get_age(context,y666_title)
            race = find_race(context)
    
        if len(y666_selftext) != 0  and race == 'unknown':
            if not y666:
                y666 = y666_selftext
            context = get_context(selftext_split,y666_selftext)[0]
            context.extend(generate_N_grams(context,2))
            if age == "unknown":
                age = get_age(context,y666_selftext)
            race = find_race(context)
                
    if len(solo_title)!=0 or len(solo_selftext)!=0 :
        if len(solo_title) !=0  and race == 'unknown':
            special_solo = solo_title
            context = get_context(title_split,solo_title)[0]
            context.extend(generate_N_grams(context,2))
            if age == "unknown":
                age = get_age(context,solo_title)
            race = find_race(context)
    
        if len(solo_selftext) != 0 and race == 'unknown':
            if not special_solo:
                special_solo = solo_selftext
            context = get_context(selftext_split,solo_selftext)[0]
            context.extend(generate_N_grams(context,2))
            if age == "unknown":
                age = get_age(context,solo_selftext)
            race = find_race(context)
            
#     print(color.YELLOW + "race: " + color.END,race)
    
#     print(trans)
#     print(binary)
#     print(female)
#     print(male1)
#     print(y1) 
#     print(testregex)
#     print(f1)
#     print(y243)
#     print(y666)
#     print(special_solo)
    
#     print("context", context)
    
    # finding age in entire text if gender is not given
    if len(context) == 0 or age == "unknown":
        age = get_age_without_context(selftext_split,title_split,age)
    
    # Converting String type to Int type
    if age != "unknown":
        age = int(age)
        
    #comments excluding first, which is by default comment to every post (Automoderator)
    comments= df['comments'][i][1:]
    
    # number of comments on a post
    num_of_cmnts = len(comments) 
#     print(color.BLUE + "Number of Comments: "+color.END, num_of_cmnts)
    
    # Posts having zero comments
    if len(comments) == 0:
        zero_cmnts_post.append(1)
        
        #to check which post number have zero comments for verification(testing)
        jiro.append(i)
        
    # posts with atleast one non OP comment    
    for x in comments:
        if x["is_submitter"].strip() == "False":
            post_num_atleast_one_nonOP_cmnt.append(1)
            
            # post numbers which have atleast 1 non op comment (testing)
            abc.append(i)
            break
            
    # number of comments by non-OP non-verified people
    number_nOP_UP_cmnts = []
    for x in comments:
        if x["author_flair_css_class"].strip() in ['default', 'None'] and x["is_submitter"].strip() == "False":
            number_nOP_UP_cmnts.append(1)
            
#     print("Number of comments by non OP non-verified people: ", len(number_nOP_UP_cmnts))
            
    # Count of posts with atleast one professional comment
    for x in comments:
        if x["author_flair_css_class"].strip() not in ['default', 'None'] and x["is_submitter"].strip() == "False":
            atleast_one_prof_nonOP_cmnt.append(1)
            break
            
    #number of comments by verified professionals
    # number of unique commenters
    # number of unique professional commenters
    verified_num_cmnts = 0
    for x in comments:
        # to see the number of unique commenters
        unique_commenters.add(x["cmnt_author"])
        
        if x["author_flair_css_class"].strip() not in ['default', 'None'] and x["is_submitter"].strip() == "False":
            # number of unique professional commenters
            unique_professionals.add(x["cmnt_author"])
            
            #number of comments by verified professionals
            verified_num_cmnts = verified_num_cmnts + 1
            
#     print(color.RED + "number of verified comments: " +color.END, verified_num_cmnts)
    
    # Number of Posts on which OP replied to alteast one of the NON-OP comments on the posts
    op_cmnt_reply = 0
    for x in comments:
        if x["is_submitter"].strip() == "True" and x["parent_id"].startswith("t1_"):
            op_cmnt_reply = op_cmnt_reply + 1
            break 
            
    # Number of comments by OP on the post
    op_cmnt_count = []
    for x in comments:
        if x["is_submitter"].strip() == "True":
            op_cmnt_count.append(1)        
#     print("Number of OP comments: ", len(op_cmnt_count))
    
    #Engagement (Ratio of users to which OP replied) None is also one type of user as some users delete their account and author name becomes null
    ratio = 0.0
    ratio_post = 0.0
    atleast_one_nop_cmnt_flag = 0
    
    # check if there are comments in the post
    if len(comments) != 0:
        
        unique_nonOP_commenters = set()
        NOP_name_cmnt_mapping = {}
        OP_nOP_reply = set()
        
        # gives name of all non-op commenters
        for x in comments:
            if x["is_submitter"].strip() == "False":
                unique_nonOP_commenters.add(x["cmnt_author"])
                
#         print(color.CYAN + "Unique Commenters: " +color.END,unique_nonOP_commenters )
        
        # this check makes sure that there is atleast one non-op comment
        if len(unique_nonOP_commenters)!= 0:
            atleast_one_nop_cmnt_flag = 1
            
            # makes non-op name as key in the dictionary and value is a list of comment id of the commnents by the author
            for i in unique_nonOP_commenters:
                NOP_name_cmnt_mapping[i] = []


            # author-comment_id mapping
            for x in comments:
                if x["is_submitter"].strip() == "False":
                    NOP_name_cmnt_mapping[x["cmnt_author"]].append(x["cmnt_id"])

#             print(color.PURPLE + "comment ID mapping: " + color.END, NOP_name_cmnt_mapping) 
            
            # to see to which Non-ops, OP have replied
            for x in comments:
                if x["is_submitter"].strip() == "True" and x["parent_id"].startswith("t1_"):
                    parent_id = x["parent_id"][3:]

                    for key,values in NOP_name_cmnt_mapping.items():
                        if parent_id in values:
                            OP_nOP_reply.add(key)
                            break
                            
            ratio = round(len(OP_nOP_reply)/len(unique_nonOP_commenters),3)
            
            # it gives 1 if there is any engagement by OP with non OP and 0 otherwise
            if ratio > 0.0:
                ratio_post = 1
                
#             print(color.PURPLE + "Users to which OP replied: " + color.END, OP_nOP_reply)
#             print(color.YELLOW + "Ratio of OP reply to unique nOP: " + color.END, ratio)
            
##Gender, Race and Age Analysis starts here

    # *************** Race Analysis ***********************************

    # to check the number of comments for different races
    if race == "latino/hispanic":
        lat_his_cmnts_list.append(num_of_cmnts)
        verified_lat_his_cmnts_list.append(verified_num_cmnts)
        lat_his_OP_list.append(op_cmnt_reply)
        if atleast_one_nop_cmnt_flag == 1:
            lat_his_engagement.append(ratio)
            lat_his_engagement_post_count.append(ratio_post)
        
    elif race == "white":
        white_cmnts_list.append(num_of_cmnts)
        verified_white_cmnts_list.append(verified_num_cmnts)
        white_OP_list.append(op_cmnt_reply)
        if atleast_one_nop_cmnt_flag == 1:
            white_engagement.append(ratio)
            white_engagement_post_count.append(ratio_post)
        
    elif race == "black":
        black_cmnts_list.append(num_of_cmnts)
        verified_black_cmnts_list.append(verified_num_cmnts)
        black_OP_list.append(op_cmnt_reply)
        if atleast_one_nop_cmnt_flag == 1:
            black_engagement.append(ratio)
            black_engagement_post_count.append(ratio_post)
        
    elif race == "asian":
        asian_cmnts_list.append(num_of_cmnts)
        verified_asian_cmnts_list.append(verified_num_cmnts)
        asian_OP_list.append(op_cmnt_reply)
        if atleast_one_nop_cmnt_flag == 1:
            asian_engagement.append(ratio)
            asian_engagement_post_count.append(ratio_post)
        
    elif race == "american indian":
        american_indian_cmnts_list.append(num_of_cmnts)
        verified_american_indian_cmnts_list.append(verified_num_cmnts)
        american_indian_OP_list.append(op_cmnt_reply)
        if atleast_one_nop_cmnt_flag == 1:
            american_indian_engagement.append(ratio)
            american_indian_engagement_post_count.append(ratio_post)
        
    elif race == "hawaiin":
        hawaiin_cmnts_list.append(num_of_cmnts)
        verified_hawaiin_cmnts_list.append(verified_num_cmnts)
        hawaiin_OP_list.append(op_cmnt_reply)
        if atleast_one_nop_cmnt_flag == 1:
            hawaiin_engagement.append(ratio)
            hawaiin_engagement_post_count.append(ratio_post)
        
    else:
        unknown_race_cmnts_list.append(num_of_cmnts)
        verified_unknown_race_cmnts_list.append(verified_num_cmnts)
        unknown_race_OP_list.append(op_cmnt_reply)
        if atleast_one_nop_cmnt_flag == 1:
            unknown_race_engagement.append(ratio)
            unknown_race_engagement_post_count.append(ratio_post)
        

    gender = "unknown"

    if len(trans) != 0:
        gender = "trans"
        
    elif len(binary) != 0:
        gender = "non-binary"

    elif (len(female) !=0) and (len(male1) !=0):
        gender = "unknown"

    elif len(female) !=0:
        gender = "female"

    elif len(male1) !=0:
        gender = "male"
        
    elif len(testregex) != 0:
        str1 = testregex[0]
        index = str1.find("m")
        if index != -1:
            gender = "male"    
        else:
            gender = "female"

    elif len(y1) != 0:
        str1 = y1[0]
        index = str1.find("m")
        if index != -1:
            gender = "male"    
        else:
            gender = "female"
            
    elif len(f1) != 0:
        str1 = f1[0]
        index = str1.find("m")
        if index != -1:
            gender = "male" 
        else:
            gender = "female"
            
    elif len(y243) != 0:
        str1 = y243[0]
        index = str1.find("m")
        if index != -1:
            gender = "male"   
        else:
            gender = "female"
            
    elif len(y666) != 0:
        str1 = y666[0]
        index = str1.find("m")
        if index != -1:
            gender = "male"   
        else:
            gender = "female"
            
    elif len(special_solo) != 0:
        str1 = special_solo[0]
        index = str1.find("m")
        if index != -1:
            gender = "male" 
        else:
            gender = "female"
            
    else:
        gender = "unknown"
    
#     print(color.BLUE + "gender: " + color.END, gender)

    # *************** Gender Analysis ***********************************
    if gender == "male":
        male_comnt_list.append(num_of_cmnts)
        verified_male_list.append(verified_num_cmnts)
        male_OP_list.append(op_cmnt_reply)
        if atleast_one_nop_cmnt_flag == 1:
            male_engagement.append(ratio)
            male_engagement_post_count.append(ratio_post)
            
    elif gender == "female":
        female_comnt_list.append(num_of_cmnts)
        verified_female_list.append(verified_num_cmnts)
        female_OP_list.append(op_cmnt_reply)
        if atleast_one_nop_cmnt_flag == 1:
            female_engagement.append(ratio)
            female_engagement_post_count.append(ratio_post)
        
    elif gender == "non-binary":
        binary_comnt_list.append(num_of_cmnts)
        verified_nbinary_list.append(verified_num_cmnts)
        nbinary_OP_list.append(op_cmnt_reply)
        if atleast_one_nop_cmnt_flag == 1:
            non_binary_engagement.append(ratio)
            non_binary_engagement_post_count.append(ratio_post)
        
    elif gender == "trans":
        trans_comnt_list.append(num_of_cmnts)
        verified_trans_list.append(verified_num_cmnts)
        trans_OP_list.append(op_cmnt_reply)
        if atleast_one_nop_cmnt_flag == 1:
            trans_engagement.append(ratio)
            trans_engagement_post_count.append(ratio_post)
        
    else:
        unknown_cmnt_list.append(num_of_cmnts)
        verified_unknown_list.append(verified_num_cmnts) 
        unknown_OP_list.append(op_cmnt_reply)
        if atleast_one_nop_cmnt_flag == 1:
            unknown_engagement.append(ratio)
            unknown_engagement_post_count.append(ratio_post)
            
    # *************** Age Analysis ***********************************
    # unknown
    if age == "unknown":
        unknown_age_cmnt_list.append(num_of_cmnts)
        verified_unknown_age_cmnt_list.append(verified_num_cmnts) 
        unknown_age_OP_list.append(op_cmnt_reply)
        if atleast_one_nop_cmnt_flag == 1:
            unknown_age_engagement.append(ratio)
            unknown_age_engagement_post_count.append(ratio_post)
    #children
    elif 0 <= age <= 12:
        child_comnt_list.append(num_of_cmnts)
        verified_child_cmnt_list.append(verified_num_cmnts)
        child_OP_list.append(op_cmnt_reply)
        if atleast_one_nop_cmnt_flag == 1:
            child_engagement.append(ratio)
            child_engagement_post_count.append(ratio_post)
            
    # teenager       
    elif 13 <= age <= 19:
        teen_comnt_list.append(num_of_cmnts)
        verified_teen_cmnt_list.append(verified_num_cmnts)
        teen_OP_list.append(op_cmnt_reply)
        if atleast_one_nop_cmnt_flag == 1:
            teen_engagement.append(ratio)
            teen_engagement_post_count.append(ratio_post)
    
    #adult
    elif 20 <= age <= 39:
        adult_comnt_list.append(num_of_cmnts)
        verified_adult_cmnt_list.append(verified_num_cmnts)
        adult_OP_list.append(op_cmnt_reply)
        if atleast_one_nop_cmnt_flag == 1:
            adult_engagement.append(ratio)
            adult_engagement_post_count.append(ratio_post)
    
    #middle age
    elif 40 <= age <= 59:
        middle_age_comnt_list.append(num_of_cmnts)
        verified_middle_age_cmnt_list.append(verified_num_cmnts)
        middle_age_OP_list.append(op_cmnt_reply)
        if atleast_one_nop_cmnt_flag == 1:
            middle_age_engagement.append(ratio)
            middle_age_engagement_post_count.append(ratio_post)
    
    # senior
    elif 60 <= age:
        senior_cmnt_list.append(num_of_cmnts)
        verified_senior_cmnt_list.append(verified_num_cmnts)
        senior_OP_list.append(op_cmnt_reply)
        if atleast_one_nop_cmnt_flag == 1:
            senior_engagement.append(ratio)
            senior_engagement_post_count.append(ratio_post)
            
#     print("age: ",age)


            
            
print("\n")
# print("Personal analysis -------------------------------")
# print("Total posts:", len(total_posts))
# print("Atleast one non OP comment: ", len(abc))
# print("Zero comments: ", len(jiro))

set_total = set(total_posts)
set_abc = set(abc)
set_jiro = set(jiro)


xyz = set_total - set_abc
only_op = xyz - set_jiro
# print("Posts where only OP commenting:", len(only_op))
# print("post number of only op commenting:", only_op)


print("\n")
print("Total Count of Posts: ", len(df))
print("Count of Posts with 0 comments: ",  len(zero_cmnts_post))
print("Posts where only OP commenting:", len(only_op))
print("Posts where atleast there is one non OP comment: ", sum(post_num_atleast_one_nonOP_cmnt))
print("Percentage of posts having atleast one non OP comment = ", (round(sum(post_num_atleast_one_nonOP_cmnt)/len(df),3))*100, "%")

unique_users = unique_posters.union(unique_commenters)
print("Count of Unique Users: ",len(unique_users))
print("Count of Unique Posters: ", len(unique_posters))
print("Count of Unique commenters: ", len(unique_commenters))
print("Count of Unique Professional Commenters: ", len(unique_professionals))

print("Posts where atleast one professional comment: ", sum(atleast_one_prof_nonOP_cmnt))
print("total comments = ",sum(male_comnt_list)+sum(female_comnt_list)+sum(trans_comnt_list)+ sum(binary_comnt_list)+sum(unknown_cmnt_list))
print("Total comments by professionals = ", sum(verified_male_list)+sum(verified_female_list)+sum(verified_trans_list)+sum(verified_nbinary_list)+sum(verified_unknown_list))

print("\n")

print("Gender Analysis ----------------------------------------------- ")
print("(a) Total Count of Posts: ", len(df))
print("(b) Self-reported demographics (gender) as extracted from posts {}:{} , {}:{}, {}:{}, {}:{}, {}:{}".format("male",len(male_comnt_list),
                                                                 "female",len(female_comnt_list),
                                                                 "trans",len(trans_comnt_list),
                                                                 "non-binary",len(binary_comnt_list),
                                                                 "unknown",len(unknown_cmnt_list)))

print("(c) Percentage (b/a) Self-reported demographics as extracted from posts {}:{} , {}:{}, {}:{}, {}:{}, {}:{}".format("male",round((len(male_comnt_list)/len(df))*100,2),
                                                                             "female",round((len(female_comnt_list)/len(df))*100,2),
                                                                             "trans",round((len(trans_comnt_list)/len(df))*100,2),
                                                                             "non-binary",round((len(binary_comnt_list)/len(df))*100,2),
                                                                             "unknown",round((len(unknown_cmnt_list)/len(df))*100,2)))

num_gender_disclose_post = len(df) - len(unknown_cmnt_list)
print("(d) Posts which disclose their gender '(a) - unknown' = ", num_gender_disclose_post)
print("(e) (b/d) Of the posts that included self-reported gender {}:{} , {}:{}, {}:{}, {}:{}".format("male",round((len(male_comnt_list)/num_gender_disclose_post)*100,2),
                                                                             "female",round((len(female_comnt_list)/num_gender_disclose_post)*100,2),
                                                                             "trans",round((len(trans_comnt_list)/num_gender_disclose_post)*100,2),
                                                                             "non-binary",round((len(binary_comnt_list)/num_gender_disclose_post)*100,2)))
                                                                                                          
print("(f) Posts where atleast there is one non OP comment: ", sum(post_num_atleast_one_nonOP_cmnt), " ratio (f/a) ",round((sum(post_num_atleast_one_nonOP_cmnt)/len(df))*100,2))
print("(g) Number of Posts where there is atleast one non op comment {}:{} , {}:{}, {}:{}, {}:{}, {}:{}".format("male",len(male_engagement),
                                                                                  "female",len(female_engagement),
                                                                                  "trans",len(trans_engagement),
                                                                                  "non-binary",len(non_binary_engagement),
                                                                                  "unknown",len(unknown_engagement)))

print("(h) Ratio (g/b) of Posts where there is atleast one non op comment {}:{} , {}:{}, {}:{}, {}:{}, {}:{}".format("male",round((len(male_engagement)/len(male_comnt_list))*100,2),
                                                                                  "female",round((len(female_engagement)/len(female_comnt_list))*100,2),
                                                                                  "trans",round((len(trans_engagement)/len(trans_comnt_list))*100,2),
                                                                                  "non-binary",round((len(non_binary_engagement)/len(binary_comnt_list))*100,2),
                                                                                  "unknown",round((len(unknown_engagement)/len(unknown_cmnt_list))*100,2)))

print("(i) Number of Posts OPs Engaging in discussion given there is atleast one non op comment {}:{} , {}:{}, {}:{}, {}:{}, {}:{}".format("male",sum(male_engagement_post_count),
                                                                                  "female",sum(female_engagement_post_count),
                                                                                  "trans",sum(trans_engagement_post_count),
                                                                                  "non-binary",sum(non_binary_engagement_post_count),
                                                                                  "unknown",sum(unknown_engagement_post_count)))

print("(j) Ratio (i/g) of Posts OPs Engaging in discussion given there is atleast one non op comment {}:{} , {}:{}, {}:{}, {}:{}, {}:{}".format("male",round((sum(male_engagement_post_count)/len(male_engagement))*100,2),
                                                                                  "female",round((sum(female_engagement_post_count)/len(female_engagement))*100,2),
                                                                                  "trans",round((sum(trans_engagement_post_count)/len(trans_engagement))*100,2),
                                                                                  "non-binary",round((sum(non_binary_engagement_post_count)/len(non_binary_engagement))*100,2),
                                                                                  "unknown",round((sum(unknown_engagement_post_count)/len(unknown_engagement))*100,2)))

print("(k) Ratio of OPs (Ratio wise for each post) Engaging in discussion given there is atleast one non op comment {}:{} , {}:{}, {}:{}, {}:{}, {}:{}".format("male",round(calculate_mean(male_engagement)*100,2),
                                                                                  "female",round(calculate_mean(female_engagement)*100,2),
                                                                                  "trans",round(calculate_mean(trans_engagement)*100,2),
                                                                                  "non-binary",round(calculate_mean(non_binary_engagement)*100,2),
                                                                                  "unknown",round(calculate_mean(unknown_engagement)*100,2)))

print("(l) Number of comments on posts posted by {}:{} , {}:{}, {}:{}, {}:{}, {}:{}".format("male",sum(male_comnt_list),
                                                                                  "female",sum(female_comnt_list),
                                                                                  "trans",sum(trans_comnt_list),
                                                                                  "non-binary",sum(binary_comnt_list),
                                                                                  "unknown",sum(unknown_cmnt_list)))

print("(m) Average (l/b) of comments on posts by {}:{} , {}:{}, {}:{}, {}:{}, {}:{}".format("male",round(calculate_mean(male_comnt_list),2),
                                                                                "female",round(calculate_mean(female_comnt_list),2),
                                                                                "trans",round(calculate_mean(trans_comnt_list),2),
                                                                                "non-binary",round(calculate_mean(binary_comnt_list),2),
                                                                                "unknown",round(calculate_mean(unknown_cmnt_list),2)))

print("(n) Average (l/g) of comments given posts that have atleast one comment by non-OP on posts by {}:{} , {}:{}, {}:{}, {}:{}, {}:{}".format("male",round(sum(male_comnt_list)/len(male_engagement),2),
                                                                                  "female",round(sum(female_comnt_list)/len(female_engagement),2),
                                                                                  "trans",round(sum(trans_comnt_list)/len(trans_engagement),2),
                                                                                  "non-binary",round(sum(binary_comnt_list)/len(non_binary_engagement),2),
                                                                                  "unknown",round(sum(unknown_cmnt_list)/len(unknown_engagement),2)))


print("(o) Number of comments by Professionals on posts posted by {}:{} , {}:{}, {}:{}, {}:{}, {}:{}".format("male",sum(verified_male_list),
                                                                           "female",sum(verified_female_list),
                                                                           "trans",sum(verified_trans_list),
                                                                           "non-binary",sum(verified_nbinary_list),
                                                                           "unknown",sum(verified_unknown_list)))

print("(p) Mean (o/b) of Professional replies received on posts posted by {}:{} , {}:{}, {}:{}, {}:{}, {}:{}".format("male",round(calculate_mean(verified_male_list),2),
                                                                          "female",round(calculate_mean(verified_female_list),2),
                                                                          "trans",round(calculate_mean(verified_trans_list),2),
                                                                          "non-binary",round(calculate_mean(verified_nbinary_list),2),
                                                                          "unknown",round(calculate_mean(verified_unknown_list),2)))

print("(q) Mean (o/g) of Professional replies received on posts given posts that have atleast one comment by non-OP {}:{} , {}:{}, {}:{}, {}:{}, {}:{}".format("male",round(sum(verified_male_list)/len(male_engagement),2),
                                                                          "female",round(sum(verified_female_list)/len(female_engagement),2),
                                                                          "trans",round(sum(verified_trans_list)/len(trans_engagement),2),
                                                                          "non-binary",round(sum(verified_nbinary_list)/len(non_binary_engagement),2),
                                                                          "unknown",round(sum(verified_unknown_list)/len(unknown_engagement),2)))

print("\n")
print("Race Analysis -----------------------------------")
print("(a) Total Count of Posts: ", len(df))
print("(b) Self-reported demographics (race) as extracted from posts   {}:{} , {}:{}, {}:{}, {}:{}, {}:{}, {}:{}, {}:{}".format("latino/hispanic",len(lat_his_cmnts_list),
                                                                               "white",len(white_cmnts_list), 
                                                                               "black",len(black_cmnts_list), 
                                                                               "asian", len(asian_cmnts_list), 
                                                                               "american indian",len(american_indian_cmnts_list),
                                                                               "hawaiin", len(hawaiin_cmnts_list),
                                                                              "unknown", len(unknown_race_cmnts_list)))

print("(c) Percentage (b/a) Self-reported demographics (race) as extracted from posts {}:{} , {}:{}, {}:{}, {}:{}, {}:{}, {}:{}, {}:{}".format("latino/hispanic",round((len(lat_his_cmnts_list)/len(df))*100,2),
                                                                               "white",round((len(white_cmnts_list)/len(df))*100,2), 
                                                                               "black",round((len(black_cmnts_list)/len(df))*100,2),
                                                                               "asian",round((len(asian_cmnts_list)/len(df))*100,2), 
                                                                               "american indian",round((len(american_indian_cmnts_list)/len(df))*100,2),
                                                                               "hawaiin",round((len(hawaiin_cmnts_list)/len(df))*100,2),
                                                                              "unknown",round((len(unknown_race_cmnts_list)/len(df))*100,2)))
num_race_disclose_post = len(df) - len(unknown_race_cmnts_list)
print("(d) Posts which disclose their race '(a) - unknown' = ", num_race_disclose_post)
print("(e) (b/d) Of the posts that included self-reported race {}:{} , {}:{}, {}:{}, {}:{}, {}:{}, {}:{}".format("latino/hispanic",round((len(lat_his_cmnts_list)/num_race_disclose_post)*100,2),
                                                                               "white",round((len(white_cmnts_list)/num_race_disclose_post)*100,2),
                                                                               "black",round((len(black_cmnts_list)/num_race_disclose_post)*100,2), 
                                                                               "asian",round((len(asian_cmnts_list)/num_race_disclose_post)*100,2),
                                                                               "american indian",round((len(american_indian_cmnts_list)/num_race_disclose_post)*100,2),
                                                                               "hawaiin",round((len(hawaiin_cmnts_list)/num_race_disclose_post)*100,2)))

print("(f) Posts where atleast there is one non OP comment: ", sum(post_num_atleast_one_nonOP_cmnt), " ratio (f/a) ",round((sum(post_num_atleast_one_nonOP_cmnt)/len(df))*100,2))
print("(g) Number of posts where there is atleast one non-OP comment {}:{} , {}:{}, {}:{}, {}:{}, {}:{}, {}:{}, {}:{}".format("latino/hispanic",len(lat_his_engagement),
                                                                               "white",len(white_engagement), 
                                                                               "black",len(black_engagement), 
                                                                               "asian", len(asian_engagement), 
                                                                               "american indian",len(american_indian_engagement),
                                                                               "hawaiin", len(hawaiin_engagement),
                                                                              "unknown", len(unknown_race_engagement)))

print("(h) Ratio (g/b) of Posts where there is atleast one non op comment {}:{} , {}:{}, {}:{}, {}:{}, {}:{}, {}:{}, {}:{}".format("latino/hispanic",round((len(lat_his_engagement)/len(lat_his_cmnts_list))*100,2),
                                                                               "white",round((len(white_engagement)/len(white_cmnts_list))*100,2),
                                                                               "black",round((len(black_engagement)/len(black_cmnts_list))*100,2),
                                                                               "asian",round((len(asian_engagement)/len(asian_cmnts_list))*100,2),
                                                                               "american indian",round((len(american_indian_engagement)/len(american_indian_cmnts_list))*100,2),
                                                                               "hawaiin",round((len(hawaiin_engagement)/len(hawaiin_cmnts_list))*100,2),
                                                                              "unknown",round((len(unknown_race_engagement)/len(unknown_race_cmnts_list))*100,2)))

print("(i) Number of Posts OPs Engaging in discussion given there is atleast one non op comment {}:{} , {}:{}, {}:{}, {}:{}, {}:{}, {}:{}, {}:{}".format("latino/hispanic",sum(lat_his_engagement_post_count),
                                                                               "white",sum(white_engagement_post_count), 
                                                                               "black",sum(black_engagement_post_count), 
                                                                               "asian", sum(asian_engagement_post_count), 
                                                                               "american indian",sum(american_indian_engagement_post_count),
                                                                               "hawaiin", sum(hawaiin_engagement_post_count),
                                                                              "unknown", sum(unknown_race_engagement_post_count)))

print("(j) Ratio (i/g) of Posts OPs Engaging in discussion given there is atleast one non op comment  {}:{} , {}:{}, {}:{}, {}:{}, {}:{}, {}:{}, {}:{}".format("latino/hispanic",round((sum(lat_his_engagement_post_count)/len(lat_his_engagement))*100,2),
                                                                               "white",round((sum(white_engagement_post_count)/len(white_engagement))*100,2), 
                                                                               "black",round((sum(black_engagement_post_count)/len(black_engagement))*100,2), 
                                                                               "asian", round((sum(asian_engagement_post_count)/len(asian_engagement))*100,2), 
                                                                               "american indian",round((sum(american_indian_engagement_post_count)/len(american_indian_engagement))*100,2),
                                                                               "hawaiin", round((sum(hawaiin_engagement_post_count)/len(hawaiin_engagement))*100,2),
                                                                              "unknown", round((sum(unknown_race_engagement_post_count)/len(unknown_race_engagement))*100,2)))

print("(k) Ratio of OPs (Ratio wise for each post) Engaging in discussion given there is atleast one non op comment {}:{} , {}:{}, {}:{}, {}:{}, {}:{}, {}:{}, {}:{}".format("latino/hispanic",round(calculate_mean(lat_his_engagement)*100,2),
                                                                               "white",round(calculate_mean(white_engagement)*100,2), 
                                                                               "black",round(calculate_mean(black_engagement)*100,2), 
                                                                               "asian",round(calculate_mean(asian_engagement)*100,2), 
                                                                               "american indian",round(calculate_mean(american_indian_engagement),2),
                                                                               "hawaiin",round(calculate_mean(hawaiin_engagement)*100,2),
                                                                              "unknown",round(calculate_mean(unknown_race_engagement)*100,2)))

print("(l) Number of comments on posts posted by {}:{} , {}:{}, {}:{}, {}:{}, {}:{}, {}:{}, {}:{}".format("latino/hispanic",sum(lat_his_cmnts_list),
                                                                               "white",sum(white_cmnts_list), 
                                                                               "black",sum(black_cmnts_list), 
                                                                               "asian", sum(asian_cmnts_list), 
                                                                               "american indian",sum(american_indian_cmnts_list),
                                                                               "hawaiin", sum(hawaiin_cmnts_list),
                                                                              "unknown", sum(unknown_race_cmnts_list)))

print("(m) Average (l/b) of comments received on posts by {}:{} , {}:{}, {}:{}, {}:{}, {}:{}, {}:{}, {}:{}".format("latino/hispanic",round(calculate_mean(lat_his_cmnts_list),2),
                                                                                "white",round(calculate_mean(white_cmnts_list),2),
                                                                                "black",round(calculate_mean(black_cmnts_list),2),
                                                                                "asian",round(calculate_mean(asian_cmnts_list),2),
                                                                                "american indian",round(calculate_mean(american_indian_cmnts_list),2),
                                                                                "hawaiin",round(calculate_mean(hawaiin_cmnts_list),2),
                                                                                "unknown",round(calculate_mean(unknown_race_cmnts_list),2)))

print("(n) Average (l/g) of comments given posts that have atleast one comment by non-OP on posts by {}:{} , {}:{}, {}:{}, {}:{}, {}:{}, {}:{}, {}:{}".format("latino/hispanic",round(sum(lat_his_cmnts_list)/len(lat_his_engagement),2),
                                                                               "white",round(sum(white_cmnts_list)/len(white_engagement),2),
                                                                               "black",round(sum(black_cmnts_list)/len(black_engagement),2),
                                                                               "asian",round(sum(asian_cmnts_list)/len(asian_engagement),2),
                                                                               "american indian",round(sum(american_indian_cmnts_list)/len(american_indian_engagement),2),
                                                                               "hawaiin",round(sum(hawaiin_cmnts_list)/len(hawaiin_engagement),2),
                                                                              "unknown",round(sum(unknown_race_cmnts_list)/len(unknown_race_engagement),2)))

print("(o) Number of comments by Professionals on posts posted by {}:{} , {}:{}, {}:{}, {}:{}, {}:{}, {}:{}, {}:{}".format("latino/hispanic",sum(verified_lat_his_cmnts_list),
                                                                                "white",sum(verified_white_cmnts_list),
                                                                                "black",sum(verified_black_cmnts_list),
                                                                                "asian",sum(verified_asian_cmnts_list),
                                                                                "american indian",sum(verified_american_indian_cmnts_list),
                                                                                "hawaiin",sum(verified_hawaiin_cmnts_list),
                                                                                "unknown",sum(verified_unknown_race_cmnts_list)))

print("(p) Mean (o/b) of Professional replies received on posts posted by {}:{} , {}:{}, {}:{}, {}:{}, {}:{}, {}:{}, {}:{}".format("latino/hispanic",round(calculate_mean(verified_lat_his_cmnts_list),2),
                                                                                "white",round(calculate_mean(verified_white_cmnts_list),2),
                                                                                "black",round(calculate_mean(verified_black_cmnts_list),2),
                                                                                "asian",round(calculate_mean(verified_asian_cmnts_list),2),
                                                                                "american indian",round(calculate_mean(verified_american_indian_cmnts_list),2),
                                                                                "hawaiin",round(calculate_mean(verified_hawaiin_cmnts_list),2),
                                                                                "unknown",round(calculate_mean(verified_unknown_race_cmnts_list),2)))

print("(q) Mean (o/g) of Professional replies received on posts given posts that have atleast one comment by non-OP {}:{} , {}:{}, {}:{}, {}:{}, {}:{}, {}:{}, {}:{}".format("latino/hispanic",round(sum(verified_lat_his_cmnts_list)/len(lat_his_engagement),2),
                                                                                "white",round(sum(verified_white_cmnts_list)/len(white_engagement),2),
                                                                                "black",round(sum(verified_black_cmnts_list)/len(black_engagement),2),
                                                                                "asian",round(sum(verified_asian_cmnts_list)/len(asian_engagement),2),
                                                                                "american indian",round(sum(verified_american_indian_cmnts_list)/len(american_indian_engagement),2),
                                                                                "hawaiin",round(sum(verified_hawaiin_cmnts_list)/len(hawaiin_engagement),2),
                                                                                "unknown",round(sum(verified_unknown_race_cmnts_list)/len(unknown_race_engagement),2)))

print("\n")
print("Age Analysis -----------------------------------")
print("(a) Total Count of Posts: ", len(df))
print("(b) Self-reported demographics (age) as extracted from posts  {}:{}, {}:{}, {}:{}, {}:{}, {}:{}, {}:{}".format("children",len(child_comnt_list),
                                                                               "teenager",len(teen_comnt_list), 
                                                                               "adult",len(adult_comnt_list), 
                                                                               "middle age", len(middle_age_comnt_list), 
                                                                               "senior",len(senior_cmnt_list),
                                                                              "unknown", len(unknown_age_cmnt_list)))

print("(c) Percentage (b/a) Self-reported demographics (age) as extracted from posts {}:{}, {}:{}, {}:{}, {}:{}, {}:{}, {}:{}".format("children",round((len(child_comnt_list)/len(df))*100,2),
                                                                               "teenager",round((len(teen_comnt_list)/len(df))*100,2), 
                                                                               "adult",round((len(adult_comnt_list)/len(df))*100,2),
                                                                               "middle age",round((len(middle_age_comnt_list)/len(df))*100,2), 
                                                                               "senior",round((len(senior_cmnt_list)/len(df))*100,2),
                                                                              "unknown",round((len(unknown_age_cmnt_list)/len(df))*100,2)))
num_age_disclose_post = len(df) - len(unknown_age_cmnt_list)
print("(d) Posts which disclose their age '(a) - unknown' = ", num_age_disclose_post)
print("(e) (b/d) Of the posts that included self-reported age {}:{}, {}:{}, {}:{}, {}:{}, {}:{}".format("children",round((len(child_comnt_list)/num_age_disclose_post)*100,2),
                                                                               "teenager",round((len(teen_comnt_list)/num_age_disclose_post)*100,2),
                                                                               "adult",round((len(adult_comnt_list)/num_age_disclose_post)*100,2), 
                                                                               "middle age",round((len(middle_age_comnt_list)/num_age_disclose_post)*100,2),
                                                                               "senior",round((len(senior_cmnt_list)/num_age_disclose_post)*100,2)))

print("(f) Posts where atleast there is one non OP comment: ", sum(post_num_atleast_one_nonOP_cmnt), " ratio (f/a) ",round((sum(post_num_atleast_one_nonOP_cmnt)/len(df))*100,2))
print("(g) Number of posts where there is atleast one non-OP comment {}:{}, {}:{}, {}:{}, {}:{}, {}:{}, {}:{}".format("children",len(child_engagement),
                                                                               "teenager",len(teen_engagement), 
                                                                               "adult",len(adult_engagement), 
                                                                               "middle age", len(middle_age_engagement), 
                                                                               "senior",len(senior_engagement),
                                                                              "unknown", len(unknown_age_engagement)))

print("(h) Ratio (g/b) of Posts where there is atleast one non op comment {}:{}, {}:{}, {}:{}, {}:{}, {}:{}, {}:{}".format("children",round((len(child_engagement)/len(child_comnt_list))*100,2),
                                                                               "teenager",round((len(teen_engagement)/len(teen_comnt_list))*100,2),
                                                                               "adult",round((len(adult_engagement)/len(adult_comnt_list))*100,2),
                                                                               "middle age",round((len(middle_age_engagement)/len(middle_age_comnt_list))*100,2),
                                                                               "senior",round((len(senior_engagement)/len(senior_cmnt_list))*100,2),
                                                                              "unknown",round((len(unknown_age_engagement)/len(unknown_age_cmnt_list))*100,2)))

print("(i) Number of Posts OPs Engaging in discussion given there is atleast one non op comment {}:{}, {}:{}, {}:{}, {}:{}, {}:{}, {}:{}".format("children",sum(child_engagement_post_count),
                                                                               "teenager",sum(teen_engagement_post_count), 
                                                                               "adult",sum(adult_engagement_post_count), 
                                                                               "middle age", sum(middle_age_engagement_post_count), 
                                                                               "senior",sum(senior_engagement_post_count),
                                                                              "unknown", sum(unknown_age_engagement_post_count)))

print("(j) Ratio (i/g) of Posts OPs Engaging in discussion given there is atleast one non op comment  {}:{}, {}:{}, {}:{}, {}:{}, {}:{}, {}:{}".format("children",round((sum(child_engagement_post_count)/len(child_engagement))*100,2),
                                                                               "teenager",round((sum(teen_engagement_post_count)/len(teen_engagement))*100,2), 
                                                                               "adult",round((sum(adult_engagement_post_count)/len(adult_engagement))*100,2), 
                                                                               "middle age", round((sum(middle_age_engagement_post_count)/len(middle_age_engagement))*100,2), 
                                                                               "senior",round((sum(senior_engagement_post_count)/len(senior_engagement))*100,2),
                                                                              "unknown", round((sum(unknown_age_engagement_post_count)/len(unknown_age_engagement))*100,2)))

print("(k) Ratio of OPs (Ratio wise for each post) Engaging in discussion given there is atleast one non op comment {}:{}, {}:{}, {}:{}, {}:{}, {}:{}, {}:{}".format("children",round(calculate_mean(child_engagement)*100,2),
                                                                               "teenager",round(calculate_mean(teen_engagement)*100,2), 
                                                                               "adult",round(calculate_mean(adult_engagement)*100,2), 
                                                                               "middle age",round(calculate_mean(middle_age_engagement)*100,2), 
                                                                               "senior",round(calculate_mean(senior_engagement),2),
                                                                              "unknown",round(calculate_mean(unknown_age_engagement)*100,2)))

print("(l) Number of comments on posts posted by {}:{}, {}:{}, {}:{}, {}:{}, {}:{}, {}:{}".format("children",sum(child_comnt_list),
                                                                               "teenager",sum(teen_comnt_list), 
                                                                               "adult",sum(adult_comnt_list), 
                                                                               "middle age", sum(middle_age_comnt_list), 
                                                                               "senior",sum(senior_cmnt_list),
                                                                              "unknown", sum(unknown_age_cmnt_list)))

print("(m) Average (l/b) of comments received on posts by {}:{}, {}:{}, {}:{}, {}:{}, {}:{}, {}:{}".format("children",round(calculate_mean(child_comnt_list),2),
                                                                                "teenager",round(calculate_mean(teen_comnt_list),2),
                                                                                "adult",round(calculate_mean(adult_comnt_list),2),
                                                                                "middle age",round(calculate_mean(middle_age_comnt_list),2),
                                                                                "senior",round(calculate_mean(senior_cmnt_list),2),
                                                                                "unknown",round(calculate_mean(unknown_age_cmnt_list),2)))

print("(n) Average (l/g) of comments given posts that have atleast one comment by non-OP on posts by {}:{}, {}:{}, {}:{}, {}:{}, {}:{}, {}:{}".format("children",round(sum(child_comnt_list)/len(child_engagement),2),
                                                                               "teenager",round(sum(teen_comnt_list)/len(teen_engagement),2),
                                                                               "adult",round(sum(adult_comnt_list)/len(adult_engagement),2),
                                                                               "middle age",round(sum(middle_age_comnt_list)/len(middle_age_engagement),2),
                                                                               "senior",round(sum(senior_cmnt_list)/len(senior_engagement),2),
                                                                              "unknown",round(sum(unknown_age_cmnt_list)/len(unknown_age_engagement),2)))

print("(o) Number of comments by Professionals on posts posted by {}:{}, {}:{}, {}:{}, {}:{}, {}:{}, {}:{}".format("children",sum(verified_child_cmnt_list),
                                                                                "teenager",sum(verified_teen_cmnt_list),
                                                                                "adult",sum(verified_adult_cmnt_list),
                                                                                "middle age",sum(verified_middle_age_cmnt_list),
                                                                                "senior",sum(verified_senior_cmnt_list),
                                                                                "unknown",sum(verified_unknown_age_cmnt_list)))

print("(p) Mean (o/b) of Professional replies received on posts posted by {}:{}, {}:{}, {}:{}, {}:{}, {}:{}, {}:{}".format("children",round(calculate_mean(verified_child_cmnt_list),2),
                                                                                "teenager",round(calculate_mean(verified_teen_cmnt_list),2),
                                                                                "adult",round(calculate_mean(verified_adult_cmnt_list),2),
                                                                                "middle age",round(calculate_mean(verified_middle_age_cmnt_list),2),
                                                                                "senior",round(calculate_mean(verified_senior_cmnt_list),2),
                                                                                "unknown",round(calculate_mean(verified_unknown_age_cmnt_list),2)))

print("(q) Mean (o/g) of Professional replies received on posts given posts that have atleast one comment by non-OP {}:{}, {}:{}, {}:{}, {}:{}, {}:{}, {}:{}".format("children",round(sum(verified_child_cmnt_list)/len(child_engagement),2),
                                                                                "teenager",round(sum(verified_teen_cmnt_list)/len(teen_engagement),2),
                                                                                "adult",round(sum(verified_adult_cmnt_list)/len(adult_engagement),2),
                                                                                "middle age",round(sum(verified_middle_age_cmnt_list)/len(middle_age_engagement),2),
                                                                                "senior",round(sum(verified_senior_cmnt_list)/len(senior_engagement),2),
                                                                                "unknown",round(sum(verified_unknown_age_cmnt_list)/len(unknown_age_engagement),2)))


<ipython-input-4-8eeb24687188>:30: DeprecationWarning: Flags not at the start of the expression '(?i)trans[\\s-]*(?i)g' (truncated)
  gender = re.findall(regex,text)
<ipython-input-4-8eeb24687188>:30: DeprecationWarning: Flags not at the start of the expression '(?i)female|(?i)woman'
  gender = re.findall(regex,text)
<ipython-input-4-8eeb24687188>:30: DeprecationWarning: Flags not at the start of the expression '(?i)(?<!fe)male|(?i)' (truncated)
  gender = re.findall(regex,text)
<ipython-input-4-8eeb24687188>:30: DeprecationWarning: Flags not at the start of the expression '(?<!\\.)[^\\d]\\d\\d[\\s.' (truncated)
  gender = re.findall(regex,text)
<ipython-input-4-8eeb24687188>:30: DeprecationWarning: Flags not at the start of the expression "(?<![a-z'\\’\\.])\\d\\d[" (truncated)
  gender = re.findall(regex,text)
<ipython-input-4-8eeb24687188>:30: DeprecationWarning: Flags not at the start of the expression '\\d\\d[\\s]*[a-zA-Z]+:\\' (truncated)
  gender = re.findall(regex,text)
<ipython





Total Count of Posts:  177850
Count of Posts with 0 comments:  86450
Posts where only OP commenting: 2350
Posts where atleast there is one non OP comment:  89050
Percentage of posts having atleast one non OP comment =  50.1 %
Count of Unique Users:  157538
Count of Unique Posters:  121485
Count of Unique commenters:  85423
Count of Unique Professional Commenters:  1641
Posts where atleast one professional comment:  62850
total comments =  426854
Total comments by professionals =  127013


Gender Analysis ----------------------------------------------- 
(a) Total Count of Posts:  177850
(b) Self-reported demographics (gender) as extracted from posts male:91284 , female:69391, trans:1411, non-binary:297, unknown:15467
(c) Percentage (b/a) Self-reported demographics as extracted from posts male:51.33 , female:39.02, trans:0.79, non-binary:0.17, unknown:8.7
(d) Posts which disclose their gender '(a) - unknown' =  162383
(e) (b/d) Of the posts that included self-reported gender male:56.